# Optimization with quadratic datasets


Uploading of the file

In [ ]:
import time
import numpy as np
from google.colab import files
#First we download this provided dataset https://github.com/dwave-examples/knapsack/blob/master/data/very_large.csv
#And then we upload it here
uploaded = files.upload()  # Opens a file upload dialog

Saving 100_WR_R_DR_25.txt to 100_WR_R_DR_25.txt


In [ ]:
file = "100_WR_R_DR_25.txt"

###Loading of the new file

In [ ]:
# First thing: we define a function which takes our dataset as a text file and returns the parameters for our model: coefficients (both quadratic and none), weights and capacity
def load_qkp_data(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Parse number of items (n)
    n = int(lines[0].strip())

    # Parse profit matrix P (n x n)
    P = np.zeros((n, n), dtype=int)
    diag_row = list(map(int, lines[1].strip().split()))
    for i in range(n):
        P[i][i] = diag_row[i]
        row = list(map(int, lines[i+2].strip().split())) #in ordine: prendo la linea di testo, rimuovo gli spazi, suddivido ogni numero in un elemento della lista, e lo converto a numero intero
        P[i][i+1:] = row

    # Parse weights and capacity
    weights = list(map(int, lines[-1].strip().split()))  # Last line is weights
    C = int(lines[-2].strip())  # Capacity is the second last value

    return P, weights, C

In [ ]:
P, w, C = load_qkp_data("100_WR_R_DR_25.txt")
print(P)
print(w)
print(C)

#Smaller dataset

I'll try to get a smaller dataset from the big one: just by taking only one quarter of the variables of the big one. We creat a function ad hoc to divide the dataset

In [ ]:
import time
import numpy as np
from google.colab import files
#First we download this provided dataset https://github.com/dwave-examples/knapsack/blob/master/data/very_large.csv
#And then we upload it here
uploaded = files.upload()  # Opens a file upload dialog

Saving 100_R_G_DR_100.txt to 100_R_G_DR_100.txt


In [ ]:
file = "100_R_G_DR_100.txt"

In [ ]:
# First thing: we define a function which takes our dataset as a text file and returns the parameters for our model: coefficients (both quadratic and none), weights and capacity
def divide_qkp_data(filename, param = 5):
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Parse number of items (n/5)
    N = int(lines[0].strip())
    n = N//param
    # Parse profit matrix P (n x n)
    P = np.zeros((n, n), dtype=int)
    diag_row = list(map(int, lines[1].strip().split()))
    for i in range(n):
        P[i][i] = diag_row[i]
        row = list(map(int, lines[i+2].strip().split())) #in ordine: prendo la linea di testo, rimuovo gli spazi, suddivido ogni numero in un elemento della lista, e lo converto a numero intero
        P[i][i+1:] = row[:n-i-1]

    # Parse weights and capacity
    weights = list(map(int, lines[-1].strip().split()))  # Last line is weights
    C = int(lines[-2].strip())
    tightness = C / sum(weights)
    weights_1 = weights [-n:]
    print(weights_1)
    C_new = int(tightness * sum(weights_1))
    #w = [(param//2)*x // param for x in weights_1]
    #C = int(lines[-2].strip())//param  # Capacity is the second last value

    return P, weights_1, C_new

P, w, C = divide_qkp_data("100_R_G_DR_100.txt",7)
print(P)
print(w)
print(C)

[117, 113, 120, 107, 117, 121, 114, 122, 123, 121, 123, 102, 100, 120]
[[119 105 109 101 110 113 104 104 112 108 104 102 108 107]
 [  0 122 111 103 112 115 106 106 114 110 106 104 110 109]
 [  0   0 114 107 115 119 110 110 118 114 110 108 114 112]
 [  0   0   0 124 108 111 102 102 110 106 102 100 106 105]
 [  0   0   0   0 114 120 111 111 119 115 111 109 115 113]
 [  0   0   0   0   0 116 114 114 122 118 114 112 118 117]
 [  0   0   0   0   0   0 108 106 113 109 106 103 109 108]
 [  0   0   0   0   0   0   0 110 113 109 106 103 109 108]
 [  0   0   0   0   0   0   0   0 109 117 113 111 117 116]
 [  0   0   0   0   0   0   0   0   0 114 109 107 114 112]
 [  0   0   0   0   0   0   0   0   0   0 107 103 109 108]
 [  0   0   0   0   0   0   0   0   0   0   0 103 107 106]
 [  0   0   0   0   0   0   0   0   0   0   0   0 111 112]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 101]]
[117, 113, 120, 107, 117, 121, 114, 122, 123, 121, 123, 102, 100, 120]
373


# Methods

###ExactPolySolver and Exact Solver

In [ ]:
!pip install dwave-ocean-sdk
import dimod
from dimod import BinaryQuadraticModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.7/103.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00


PolySolver

In [ ]:
from dimod import ExactPolySolver
import numpy as np
def solve_qkp_with_polysolver(P, w, C, lambda_penalty=None):
    n = len(w)

    if lambda_penalty is None:
        lambda_penalty = np.max(P)*10
    #Slack variables (computing the coefficients)
    max_slack = C - min(w)
    slack_bits = int(np.ceil(np.log2(max_slack + 1)))  # n° di bit per rappresentare lo slack
    z_weights = [1 << i for i in range(slack_bits)]  # es: [1, 2, 4]

    # Build BQM as Polynomial
    bqm = BinaryQuadraticModel("BINARY")
    # Profit terms
    for i in range(n):
      bqm.add_variable(f"x_{i}", -P[i, i])  # Linear terms
      for j in range(i + 1, n):  # Upper triangle only
          bqm.add_interaction(f"x_{i}", f"x_{j}", -P[i, j])  # Quadratic terms (already symmetric)

    # Penalty terms
    for i in range(n):
        bqm.add_variable(f"x_{i}", lambda_penalty * (w[i]*w[i] - 2*w[i]* C))

    for k in range(slack_bits):
      bqm.add_variable(f"z_{k}", lambda_penalty * (z_weights[k]*z_weights[k] - 2*z_weights[k]* C))

    for i in range(n):
        for j in range(i + 1, n):
            bqm.add_interaction(f"x_{i}", f"x_{j}", 2 * lambda_penalty * w[i] * w[j])

    for k in range(slack_bits):
      for h in range(k + 1, slack_bits):
          bqm.add_interaction(f"z_{k}", f"z_{h}", 2 * lambda_penalty * z_weights[k]* z_weights[h])

    for i in range(n):
      for k in range(slack_bits):
            bqm.add_interaction(f"x_{i}", f"z_{k}", 2 * lambda_penalty * w[i] * z_weights[k])

        # Use ExactSolver
    #sampler = ExactSolver()
    #sampleset = sampler.sample(bqm)
    sampler = ExactPolySolver()

    start_time = time.time()
    sampleset = sampler.sample_poly(bqm)
    runtime = time.time() - start_time

    best_sample = sampleset.first.sample
    best_energy = sampleset.first.energy

    selected_items = [int(var.split('_')[1]) for var, val in best_sample.items() if val == 1 and var.startswith("x_")]
    #print(best_sample.items())
    total_weight = sum(w[i] for i in selected_items)
    total_profit = sum(P[i, j] for i in selected_items for j in selected_items)

    return selected_items, total_weight, total_profit, runtime, best_energy

In [ ]:
# For small problems (up to ~20 variables)
selected_items, total_weight, total_profit, runtime, best_energy = solve_qkp_with_polysolver(P, w, C)

print("Selected items:", selected_items)
print("Total weight:", total_weight, "/", C)
print("Total profit:", total_profit)
print("Feasible?", "Yes" if total_weight <= C else "No")
print("Best energy:", best_energy)
print("Runtime:", runtime)

runtimes = np.zeros(10)
for i in range(10):
  _, _, _, runtimes[i], _ = solve_qkp_with_polysolver(P, w, C)

print(runtimes)
print("Average runtime:", np.mean(runtimes))
print("Standard deviation of runtimes:", np.std(runtimes))

Selected items: [0, 1, 10, 11, 13, 2, 3, 4, 5, 6, 7, 8, 9]
Total weight: 3155 / 3421
Total profit: 28763
Feasible? Yes
Best energy: -58282168943.0
Runtime: 126.93321228027344


In [ ]:
runtimes = np.zeros(10)
for i in range(10):
  _, _, _, runtimes[i], _ = solve_qkp_with_polysolver(P, w, C)

print(runtimes)
print("Average runtime:", np.mean(runtimes))
print("Standard deviation of runtimes:", np.std(runtimes))

[145.79495811 138.3217144  136.66533732 129.75007677 119.8190403
 120.12439728 120.34836054 120.93010139 120.43410277 122.03932452]
Average runtime: 127.42274134159088
Standard deviation of runtimes: 9.100953862906984


ExactSolver

In [ ]:
from dimod.reference.samplers import ExactSolver
import numpy as np
def solve_qkp_with_exactsolver(P, w, C, lambda_penalty=None):
    n = len(w)

    if lambda_penalty is None:
        lambda_penalty = np.max(P)*10
    #Slack variables (computing the coefficients)
    max_slack = C - min(w)
    slack_bits = int(np.ceil(np.log2(max_slack + 1)))  # n° di bit per rappresentare lo slack
    z_weights = [1 << i for i in range(slack_bits)]  # es: [1, 2, 4]

    # Build BQM as Polynomial
    bqm = BinaryQuadraticModel("BINARY")
    # Profit terms
    for i in range(n):
      bqm.add_variable(f"x_{i}", -P[i, i])  # Linear terms
      for j in range(i + 1, n):  # Upper triangle only
          bqm.add_interaction(f"x_{i}", f"x_{j}", -P[i, j])  # Quadratic terms (already symmetric)

    # Penalty terms
    for i in range(n):
        bqm.add_variable(f"x_{i}", lambda_penalty * (w[i]*w[i] - 2*w[i]* C))

    for k in range(slack_bits):
      bqm.add_variable(f"z_{k}", lambda_penalty * (z_weights[k]*z_weights[k] - 2*z_weights[k]* C))

    for i in range(n):
        for j in range(i + 1, n):
            bqm.add_interaction(f"x_{i}", f"x_{j}", 2 * lambda_penalty * w[i] * w[j])

    for k in range(slack_bits):
      for h in range(k + 1, slack_bits):
          bqm.add_interaction(f"z_{k}", f"z_{h}", 2 * lambda_penalty * z_weights[k]* z_weights[h])

    for i in range(n):
      for k in range(slack_bits):
            bqm.add_interaction(f"x_{i}", f"z_{k}", 2 * lambda_penalty * w[i] * z_weights[k])

    # Use ExactSolver
    #print(bqm)
    sampler = ExactSolver()

    start_time = time.time()
    sampleset = sampler.sample(bqm)
    runtime = time.time() - start_time
    #print(sampleset)

    best_sample = sampleset.first.sample
    best_energy = sampleset.first.energy

    selected_items = [int(var.split('_')[1]) for var, val in best_sample.items() if val == 1 and var.startswith("x_")]
    print(selected_items)
    total_weight = sum(w[i] for i in selected_items)
    total_profit = sum(P[i, j] for i in selected_items for j in selected_items)

    return selected_items, total_weight, total_profit, runtime, best_energy

In [ ]:
# For small problems (up to ~20 variables)
selected_items, total_weight, total_profit, runtime, best_energy = solve_qkp_with_exactsolver(P, w, C)

print("Selected items:", selected_items)
print("Total weight:", total_weight, "/", C)
print("Total profit:", total_profit)
print("Feasible?", "Yes" if total_weight <= C else "No")
print("Best energy:", best_energy)
print("Runtime:", runtime)

runtimes = np.zeros(10)

print(runtimes)
for i in range(10):
  _, _, _, runtimes[i], _ = solve_qkp_with_polysolver(P, w, C)

mean = np.mean(runtimes)
std = np.std(runtimes)
print("Average runtime:", mean)
print("Standard deviation of runtimes:", std)


NameError: name 'solve_qkp_with_exactsolver' is not defined

###Other way to obtain a dataset

In [ ]:
#Other possible ways to generate the dataset
bqm = dimod.generators.gnp_random_bqm(20, 0.5, 'BINARY')
    # it generates a dataset randomly with a given probability 0.5 to have quadratic interactions
bqm2 = dimod.generators.knapsack(100) #fare prova

NameError: name 'dimod' is not defined

### Simulated Annealing and Tabu Search

In [ ]:
from dwave.samplers import SimulatedAnnealingSampler
from dwave.samplers import TabuSampler

Meaning of the inputs: First, we recall that β = 1/T, where T is the temperature. Start the annealing at β = 0.1 (left extreme of the interval) (high temperature and high randomness) End the annealing at β = 4.2 (right extreme of the interval) (low temperature, focused on low energy)

beta_schedule_type='linear': Linearly increase β from 0.1 to 4.2 over the annealing process (i.e., gradually cool the system). Another possible option could have been 'geometric'.

num_sweeps: Each sweep is one full pass over all variables (flipping them one by one to reduce energy). More sweeps → longer and deeper local search → potentially better-quality solutions.

num_reads: number of independent solution attempts

In [ ]:
# SIMULATED ANNEALING
def solve_qkp_with_SA(P, w, C, lambda_penalty=None, num_reads=1000000):
    n = len(w)

    if lambda_penalty is None:
        lambda_penalty = np.max(P)*100
    #Slack variables (computing the coefficients)
    max_slack = C - min(w)
    slack_bits = int(np.ceil(np.log2(max_slack + 1)))  # n° di bit per rappresentare lo slack
    z_weights = [1 << i for i in range(slack_bits)]  # es: [1, 2, 4]

    # Build BQM as Polynomial
    bqm = BinaryQuadraticModel("BINARY")
    # Profit terms
    for i in range(n):
      bqm.add_variable(f"x_{i}", -P[i, i])  # Linear terms
      for j in range(i + 1, n):  # Upper triangle only
          bqm.add_interaction(f"x_{i}", f"x_{j}", -P[i, j])  # Quadratic terms (already symmetric)

    # Penalty terms
    for i in range(n):
        bqm.add_variable(f"x_{i}", lambda_penalty * (w[i]*w[i] - 2*w[i]* C))

    for k in range(slack_bits):
      bqm.add_variable(f"z_{k}", lambda_penalty * (z_weights[k]*z_weights[k] - 2*z_weights[k]* C))

    for i in range(n):
        for j in range(i + 1, n):
            bqm.add_interaction(f"x_{i}", f"x_{j}", 2 * lambda_penalty * w[i] * w[j])

    for k in range(slack_bits):
      for h in range(k + 1, slack_bits):
          bqm.add_interaction(f"z_{k}", f"z_{h}", 2 * lambda_penalty * z_weights[k]* z_weights[h])

    for i in range(n):
      for k in range(slack_bits):
            bqm.add_interaction(f"x_{i}", f"z_{k}", 2 * lambda_penalty * w[i] * z_weights[k])


    # Run Simulated Annealing
    sampler = SimulatedAnnealingSampler()

    start_time = time.time()
    sampleset =  sampler.sample(
    bqm,
    num_reads=500000,
    #beta_range=[0.1, 5.0],
    beta_schedule_type='linear',
    num_sweeps=5000)
    runtime = time.time() - start_time

    # Get best solution
    best_sample = sampleset.first.sample
    best_energy = sampleset.first.energy
    selected_items = [int(var.split('_')[1]) for var, val in best_sample.items() if val == 1 and var.startswith("x_")]
    total_weight = sum(w[i] for i in selected_items)
    total_profit = sum(P[i, j] for i in selected_items for j in selected_items)

    return selected_items, total_weight, total_profit, runtime, best_energy

In [ ]:
# Solve QKP with Simulated Annealing
selected_items, total_weight, total_profit, runtime, best_energy = solve_qkp_with_SA(P, w, C, num_reads=5000)

# Output results
print("Selected items (indices):", selected_items)
print("Total weight:", total_weight, "/", C)
print("Total profit:", total_profit)
print("Feasible?", "Yes" if total_weight <= C else "No")
print("Runtime:", runtime, "seconds")
print("Best energy:", best_energy)


Selected items (indices): [67, 68, 7, 70, 72, 77, 8, 82, 83, 85, 86, 87, 90, 93, 95]
Total weight: 1585 / 1588
Total profit: 5880
Feasible? Yes
Runtime: 586.8290064334869 seconds
Best energy: -30765282680.0


In [ ]:
runtimes = np.zeros(10)
for i in range(10):
  _, _, _, runtimes[i], _ = solve_qkp_with_SA(P, w, C)

mean = np.mean(runtimes)
std = np.std(runtimes)
print("Average runtime:", mean)
print("Standard deviation of runtimes:", std)

Average runtime: 1.5461336612701415
Standard deviation of runtimes: 0.183749407746142


In [ ]:
# TABU SEARCH
def solve_qkp_with_TS(P, w, C, lambda_penalty=None, num_reads=10000, tenure=1):
    n = len(w)

    if lambda_penalty is None:
        lambda_penalty = np.max(P)*100
    #Slack variables (computing the coefficients)
    max_slack = C - min(w)
    slack_bits = int(np.ceil(np.log2(max_slack + 1)))  # n° di bit per rappresentare lo slack
    z_weights = [1 << i for i in range(slack_bits)]  # es: [1, 2, 4]

    # Build BQM as Polynomial
    bqm = BinaryQuadraticModel("BINARY")
    # Profit terms
    for i in range(n):
      bqm.add_variable(f"x_{i}", -P[i, i])  # Linear terms
      for j in range(i + 1, n):  # Upper triangle only
          bqm.add_interaction(f"x_{i}", f"x_{j}", -P[i, j])  # Quadratic terms (already symmetric)

    # Penalty terms
    for i in range(n):
        bqm.add_variable(f"x_{i}", lambda_penalty * (w[i]*w[i] - 2*w[i]* C))

    for k in range(slack_bits):
      bqm.add_variable(f"z_{k}", lambda_penalty * (z_weights[k]*z_weights[k] - 2*z_weights[k]* C))

    for i in range(n):
        for j in range(i + 1, n):
            bqm.add_interaction(f"x_{i}", f"x_{j}", 2 * lambda_penalty * w[i] * w[j])

    for k in range(slack_bits):
      for h in range(k + 1, slack_bits):
          bqm.add_interaction(f"z_{k}", f"z_{h}", 2 * lambda_penalty * z_weights[k]* z_weights[h])

    for i in range(n):
      for k in range(slack_bits):
            bqm.add_interaction(f"x_{i}", f"z_{k}", 2 * lambda_penalty * w[i] * z_weights[k])

    # Run Simulated Annealing
    sampler = TabuSampler()

    start_time = time.time()
    sampleset = sampler.sample(bqm, num_reads= num_reads, tenure= tenure, initial_states=None, timeout=None)
    runtime = time.time() - start_time

    # Get best solution
    best_sample = sampleset.first.sample
    best_energy = sampleset.first.energy
    selected_items = [int(var.split('_')[1]) for var, val in best_sample.items() if val == 1]
    total_weight = sum(w[i] for i in selected_items)
    total_profit = sum(P[i, j] for i in selected_items for j in selected_items)

    return selected_items, total_weight, total_profit, runtime, best_energy

In [ ]:
# Solve QKP with TabuSearch
selected_items, total_weight, total_profit, runtime, best_energy = solve_qkp_with_TS(P, w, C, num_reads=100)

# Output results
print("Selected items (indices):", selected_items)
print("Total weight:", total_weight, "/", C)
print("Total profit:", total_profit)
print("Feasible?", "Yes" if total_weight <= C else "No")
print("Runtime:", runtime, "seconds")
print("Best energy:", best_energy)

runtimes = np.zeros(10)

for i in range(10):
  _, _, _, runtimes[i], _ = solve_qkp_with_TS(P, w, C, num_read = 100)

mean = np.mean(runtimes)
std = np.std(runtimes)
print("Average runtime:", mean)
print("Standard deviation of runtimes:", std)

### Gurobi

In [ ]:
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Solving using Gurobi
def solve_qkp_with_Gurobi(P, w, C):

    # Create Gurobi model
    model = gp.Model("QKP")
    model.setParam('OutputFlag', 1)  # Enable solver output: very useful to obtain infos while solving
    #model.setParam('TimeLimit', 600)  # Limit to 10 minutes (adjust as needed)
    #model.setParam('MIPGap', 0.75)   # Accept 5% optimality gap (faster termination)
    model.setParam('Threads', 8)     # Use more CPU cores if available
    model.setParam('Presolve', 2)       # Aggressive presolve
    model.setParam('Heuristics', 0.5)   # Spend more time on heuristics
    model.setParam('Cuts', 2)           # Generate aggressive cuts

    n = len(w)
    # Add binary variables
    x = model.addVars(n, vtype=GRB.BINARY, name="x")
    # Set objective: maximize sum_{i,j} P[i][j] * x[i] * x[j]
    objective = gp.QuadExpr() #Necessary for a quadratic expression
    for i in range(n):
        for j in range(n):
            if P[i][j] != 0:  # Skip zero terms for efficiency
                objective += P[i][j] * x[i] * x[j]
    model.setObjective(objective, GRB.MAXIMIZE)
    # Add knapsack constraint: sum_i w[i] * x[i] <= C
    model.addConstr(gp.quicksum(w[i] * x[i] for i in range(n)) <= C, "KnapsackConstraint")

    # Start solution
    x_start = [0] * n
    for i in sorted(range(n), key=lambda i: -P[i][i]/w[i]):
      if w[i] <= C:
        x_start[i] = 1
        C -= w[i]
    # Set the starting solution
    for i in range(n):
      x[i].Start = x_start[i]

    # Optimize the model
    start_time = time.time()
    model.optimize()
    runtime = time.time() - start_time

    # Extract solution
    if model.status == GRB.OPTIMAL:
        selected_items = [i for i in range(n) if x[i].X > 0.5]
        total_weight = sum(w[i] for i in selected_items)
        total_profit = model.ObjVal
       # total_energy = sum(P[i][j] for i in selected_items for j in selected_items)
        return selected_items, total_weight, total_profit, runtime# total_energy
    else:
        raise Exception("No optimal solution found.")

In [ ]:
# Solve QKP with Gurobi
selected_items, total_weight, total_profit, runtime = solve_qkp_with_Gurobi(P, w, C)

# Print results
print("Selected items (indices):", selected_items)
print("Total weight:", total_weight, "/", C)
print("Total profit:", total_profit)
print("Feasible?", "Yes" if total_weight <= C else "No")
print("Runtime:", runtime, "seconds")

Set parameter OutputFlag to value 1
Set parameter Threads to value 8
Set parameter Presolve to value 2
Set parameter Heuristics to value 0.5
Set parameter Cuts to value 2
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 1 physical cores, 2 logical processors, using up to 8 threads

         Reduce the value of the Threads parameter to improve performance


Non-default parameters:
Heuristics  0.5
Cuts  2
Presolve  2
Threads  8

Optimize a model with 1 rows, 6 columns and 6 nonzeros
Model fingerprint: 0x24af6b0c
Model has 17 quadratic objective terms
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [2e+00, 7e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]

Loaded user MIP start with objective 71

Presolve time: 0.00s
Presolv

Remark: from the test mad on this small dataset, it seems Gurobi find a better result, in terms of profit, than the other 2 methods: ExactPolySolver and SA. Why? It could be for the presence of big penalties in the 2 methods?

In [ ]:
runtimes = np.zeros(10)
for i in range(10):
  _, _, _, runtimes[i]= solve_qkp_with_Gurobi(P, w, C)

mean = np.mean(runtimes)
std = np.std(runtimes)
print("Average runtime:", mean)
print("Standard deviation of runtimes:", std)

Set parameter OutputFlag to value 1
Set parameter Threads to value 8
Set parameter Presolve to value 2
Set parameter Heuristics to value 0.5
Set parameter Cuts to value 2
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 1 physical cores, 2 logical processors, using up to 8 threads

         Reduce the value of the Threads parameter to improve performance


Non-default parameters:
Heuristics  0.5
Cuts  2
Presolve  2
Threads  8

Optimize a model with 1 rows, 14 columns and 14 nonzeros
Model fingerprint: 0x897fc691
Model has 105 quadratic objective terms
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+02, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+02, 4e+02]

Loaded user MIP start with objective 676

Presolve time: 0.00s
P

##QAOA (with cudaQ)


In [ ]:
!pip install cudaq

In [ ]:
import numpy as np
import cudaq
from scipy.optimize import minimize
import time
from cudaq import spin
from typing import List

In [ ]:
def compute_qkp_coefficients(P, w, C, lambda_penalty):
    n = len(w)  # Number of items
    slack_bits = int(np.ceil(np.log2(C + 1)))  # Bits needed for slack
    z_weights = [2**k for k in range(slack_bits)]  # Slack bit weights [1, 2, 4, ...]

    # --- Linear coefficients ---
    coeff_linear_x = [
        float(-P[i, i] + lambda_penalty * w[i] * (w[i] - 2 * C))
        for i in range(n)
    ]
    coeff_linear_s = [
        float(lambda_penalty * ( (2**k)**2 - 2 * (2**k) * C ))
        for k in range(slack_bits)
    ]

    # --- Quadratic coefficients ---
    coeff_quad_xx = []  # Item-item terms (i < j)
    for i in range(n):
        for j in range(i + 1, n):
            coeff_quad_xx.append(float(-P[i, j] + 2 * lambda_penalty * w[i] * w[j]))

    coeff_quad_xs = []  # Item-slack terms (all i, all k)
    for i in range(n):
        for k in range(slack_bits):
            coeff_quad_xs.append(float(2 * lambda_penalty * w[i] * (2**k)))

    coeff_quad_ss = []  # Slack-slack terms (k < h)
    for k in range(slack_bits):
        for h in range(k + 1, slack_bits):
            coeff_quad_ss.append(float(2 * lambda_penalty * (2**k) * (2**h)))

    return {
        "linear_x": coeff_linear_x,
        "linear_s": coeff_linear_s,
        "quad_xx": coeff_quad_xx,
        "quad_xs": coeff_quad_xs,
        "quad_ss": coeff_quad_ss,
    }

Small Datasets

In [ ]:
#Dimension = 4
C = 7
w = [2, 3, 4, 5]
P = np.array([
    [10, 10, 4, 0],
    [0, 8, 6, 2],
    [0, 0, 6, 8],
    [0, 0, 0, 7]
])
print(P)


[[10 10  4  0]
 [ 0  8  6  2]
 [ 0  0  6  8]
 [ 0  0  0  7]]


In [ ]:
#Dimension = 5
C = 9
w = [3, 2, 3, 4, 5]
P = np.array([
    [7, 4, 6, 7, 0],
    [0, 10, 10, 4, 0],
    [0, 0, 8, 6, 2],
    [0, 0, 0, 6, 8],
    [0, 0, 0, 0, 7]
])
print(P)

[[ 7  4  6  7  0]
 [ 0 10 10  4  0]
 [ 0  0  8  6  2]
 [ 0  0  0  6  8]
 [ 0  0  0  0  7]]


In [ ]:
#Dimension = 6
C = 11
w = [4, 3, 2, 3, 4, 5]
P = np.array([
    [9, 11, 6, 8, 5, 0],
    [0, 7, 4, 6, 7, 0],
    [0, 0, 10, 10, 4, 0],
    [0, 0, 0, 8, 6, 2],
    [0, 0, 0, 0, 6, 8],
    [0, 0, 0, 0, 0, 7]
])
print(P)

[[ 9 11  6  8  5  0]
 [ 0  7  4  6  7  0]
 [ 0  0 10 10  4  0]
 [ 0  0  0  8  6  2]
 [ 0  0  0  0  6  8]
 [ 0  0  0  0  0  7]]


In [ ]:
#Dimension = 6
C = 12
w = [2, 3, 4, 5, 6, 7]

P = np.array([
    [15, 5,  3,  2,  0,  0],
    [0,  20, 6,  4,  1,  0],
    [0,  0,  18, 5,  3,  0],
    [0,  0,  0,  25, 8,  4],
    [0,  0,  0,  0,  12, 6],
    [0,  0,  0,  0,  0,  10]
])

In [ ]:
#Dimension = 8
C = 20
w = [3, 4, 5, 5, 6, 7, 8, 9]

P = np.array([
    [20, 8,  6,  5,  0,  0,  0,  0],
    [0,  25, 10, 7,  4,  0,  0,  0],
    [0,  0,  30, 12, 8,  5,  0,  0],
    [0,  0,  0,  35, 15, 9,  6,  0],
    [0,  0,  0,  0,  18, 10, 7,  4],
    [0,  0,  0,  0,  0,  22, 12, 8],
    [0,  0,  0,  0,  0,  0,  15, 10],
    [0,  0,  0,  0,  0,  0,  0,  12]
])

In [ ]:
lambda_penalty = np.max(P) *35 +1

In [ ]:
# Precompute all coefficients
coefficients = compute_qkp_coefficients(P, w, C, lambda_penalty)
print(coefficients)

{'linear_x': [-38559.0, -55208.0, -70098.0, -83245.0, -94620.0, -104254.0], 'linear_s': [-20148.0, -38544.0, -70080.0, -112128.0], 'quad_xx': [10507.0, 14013.0, 17518.0, 21024.0, 24528.0, 21018.0, 26276.0, 31535.0, 36792.0, 35035.0, 42045.0, 49056.0, 52552.0, 61316.0, 73578.0], 'quad_xs': [3504.0, 7008.0, 14016.0, 28032.0, 5256.0, 10512.0, 21024.0, 42048.0, 7008.0, 14016.0, 28032.0, 56064.0, 8760.0, 17520.0, 35040.0, 70080.0, 10512.0, 21024.0, 42048.0, 84096.0, 12264.0, 24528.0, 49056.0, 98112.0], 'quad_ss': [3504.0, 7008.0, 14016.0, 14016.0, 28032.0, 56064.0]}


In [ ]:
def solve_qkp_with_QAOA(P, w, C, coefficients, lambda_penalty=lambda_penalty,p=1, num_shots=5000):
  n = len(w)
  #lambda_penalty = np.max(P) * 1000  # Heuristic: ensure penalty dominates
  # Slack variables setup
  max_slack = max(C - min(w), 0)  # Prevent negative values
  slack_bits = int(np.ceil(np.log2(max_slack + 1)))
  z_weights = [2**i for i in range(slack_bits)]  # Binary weights for slack
  coefficients = compute_qkp_coefficients(P, w, C, lambda_penalty)
  linear_x = coefficients["linear_x"]
  linear_s = coefficients["linear_s"]
  quad_xx = coefficients["quad_xx"]
  quad_xs = coefficients["quad_xs"]
  quad_ss = coefficients["quad_ss"]

  # Total qubits: n (items) + slack_bits
  total_qubits = n + slack_bits

  # Build the Hamiltonian
  hamiltonian = cudaq.SpinOperator()

  for i in range(n):
    hamiltonian += linear_x[i] * spin.z(i)

  for k in range(slack_bits):
    hamiltonian += linear_s[k] * spin.z(n + k)

  index = 0
  for i in range(n):
    for j in range(i + 1, n):
      hamiltonian += quad_xx[index] * spin.z(i) * spin.z(j)
      index += 1

  index = 0
  for i in range(n):
    for k in range(slack_bits):
      hamiltonian += quad_xs[index] * spin.z(i) * spin.z(n+k)
      index += 1

  index = 0
  for k in range(slack_bits):
    for h in range(k + 1, slack_bits):
      hamiltonian +=  quad_ss[index]* spin.z(n + k) * spin.z(n + h)
      index += 1
  def fourier_init(p, freq=1):
    t = np.arange(1, p + 1)
    gamma = np.pi / 4 + 0.1 * np.sin(freq * np.pi * t / p)
    beta  = np.pi / 8 + 0.1 * np.cos(freq * np.pi * t / p)
    return np.concatenate([gamma, beta])
  def problem_informed_init(p):
    gamma = np.full(p, 0.3 * np.sqrt(C / (np.mean(w) * len(w))))
    beta = np.full(p, np.pi/4)
    return np.concatenate([gamma, beta])
  @cudaq.kernel
  def qaoa_ansatz(
      gammas: list[float],
      betas: list[float],
      linear_x: list[float],
      linear_s: list[float],
      quad_xx: list[float],
      quad_xs: list[float],
      quad_ss: list[float],
      p: int,
      ):
    q = cudaq.qvector(total_qubits)

    # Initial Hadamard layer
    h(q)

    for layer in range(p):
      gamma = gammas[layer]
      beta = betas[layer]

      #  Cost Unitary (exp(-iγH))
      # 1. Linear terms (x_i)
      for i in range(n):
        rz(2.0 * gamma * linear_x[i], q[i])

      # 2. Linear terms (s_k)
      for k in range(slack_bits):
        rz(2.0 * gamma * linear_s[k], q[n + k])

      # 3. Quadratic terms (x_i x_j)
      idx = 0
      for i in range(n):
        for j in range(i + 1, n):
          cx(q[i], q[j])
          rz(2.0 * gamma * quad_xx[idx], q[j])
          cx(q[i], q[j])
          idx += 1

      # 4. Quadratic terms (x_i s_k)
      idx = 0
      for i in range(n):
        for k in range(slack_bits):
          cx(q[i], q[n + k])
          rz(2.0 * gamma * quad_xs[idx], q[n + k])
          cx(q[i], q[n + k])
          idx += 1

      # 5. Quadratic terms (s_k s_h)
      idx = 0
      for k in range(slack_bits):
        for h in range(k + 1, slack_bits):
          cx(q[n + k], q[n + h])
          rz(2.0 * gamma * quad_ss[idx], q[n + h])
          cx(q[n + k], q[n + h])
          idx += 1

      # --- Mixer Unitary (exp(-iβΣX)) ---
      for i in range(total_qubits):
        rx(2.0 * beta, q[i])

  def expectation(params):
    # Split params into gammas and betas
    gammas = params[:p]
    betas = params[p:]
    return cudaq.observe(
        qaoa_ansatz,
        hamiltonian,  # Your pre-built SpinOperator
        gammas,
        betas,
        coefficients["linear_x"],
        coefficients["linear_s"],
        coefficients["quad_xx"],
        coefficients["quad_xs"],
        coefficients["quad_ss"],
        p
    ).expectation()
    # Initial parameters
  #init_params_1 = np.random.uniform(0, np.pi, 2 * p)
  gamma = np.full(p, 0.85)
  beta  = np.full(p, 0.3)
  init_params_1 = np.concatenate([gamma,beta])
  init_params_2 = np.random.uniform(0, np.pi, 2 * p)
  init_params_3 = fourier_init(p, freq=1)
  init_params_4 = problem_informed_init(p)
  init_params = init_params_4

  avg_profit = np.mean(np.diag(P))
  avg_weight = np.mean(w)
  gamma_init = 0.5 * np.arcsin(np.sqrt(C / (avg_weight * len(w))))  # ~0.2-0.5
  beta_init = np.pi / 4  # Maximal mixing
  #init_params = np.concatenate([np.full(p, gamma_init), np.full(p, beta_init)])
  print("Initial parameters:", init_params)

  start_time = time.time()
  result = minimize(expectation, init_params, method='COBYLA')
  runtime = time.time() - start_time

  #samples = cudaq.sample(qaoa_ansatz, result.x[0], result.x[1], shots = num_shots )
  # Split the optimized parameters into gammas and betas
  optimized_gammas = result.x[:p]
  optimized_betas = result.x[p:]

  samples = cudaq.sample(qaoa_ansatz, optimized_gammas, optimized_betas, coefficients["linear_x"],
        coefficients["linear_s"],
        coefficients["quad_xx"],
        coefficients["quad_xs"],
        coefficients["quad_ss"],
        p,
        shots_count=num_shots,)
  # Process results
  best_sample = max(samples.items(), key=lambda x: x[1])[0]
  selected_indices = [i for i, bit in enumerate(best_sample) if bit == '1']
  new_indices = [i for i in selected_indices if i < len(w)]
  total_weight = sum(w[i] for i in new_indices)
  total_profit = sum(P[i][j] for i in new_indices for j in new_indices)

  return new_indices, total_weight, total_profit, runtime, result.fun

In [ ]:
selected_indices, total_weight, total_profit, runtime, best_energy= solve_qkp_with_QAOA(P, w, C, coefficients, p = 2)

print("Selected items:", selected_indices)
print(f"Total weight: {total_weight} / {C} ({'Feasible' if total_weight <= C else 'Infeasible'})")
print("Total profit:", total_profit)
print(f"Runtime: {runtime:.2f} seconds")
print("Best energy:", best_energy)

Initial parameters: [0.2        0.2        0.78539816 0.78539816]
Selected items: [4]
Total weight: 6 / 12 (Feasible)
Total profit: 12
Runtime: 0.29 seconds
Best energy: 77631.52360227949


# QAOA già fornito


In [ ]:
!pip install cudaq
import cudaq
from cudaq import spin

from typing import List

import numpy as np

# Here we build up a kernel for QAOA with `p` layers, with each layer
# containing the alternating set of unitaries corresponding to the problem
# and the mixer Hamiltonians. The algorithm leverages the VQE algorithm
# to compute the Max-Cut of a rectangular graph illustrated below.

#       v0  0---------------------0 v1
#           |                     |
#           |                     |
#           |                     |
#           |                     |
#       v3  0---------------------0 v2
# The Max-Cut for this problem is 0101 or 1010.

# The problem Hamiltonian
hamiltonian = 0.5 * spin.z(0) * spin.z(1) + 0.5 * spin.z(1) * spin.z(2) \
       + 0.5 * spin.z(0) * spin.z(3) + 0.5 * spin.z(2) * spin.z(3)

# Problem parameters.
qubit_count: int = 4
layer_count: int = 2
parameter_count: int = 2 * layer_count


@cudaq.kernel
def kernel_qaoa(qubit_count: int, layer_count: int, thetas: List[float]):
    """QAOA ansatz for Max-Cut"""
    qvector = cudaq.qvector(qubit_count)

    # Create superposition
    h(qvector)

    # Loop over the layers
    for layer in range(layer_count):
        # Loop over the qubits
        # Problem unitary
        for qubit in range(qubit_count):
            x.ctrl(qvector[qubit], qvector[(qubit + 1) % qubit_count])
            rz(2.0 * thetas[layer], qvector[(qubit + 1) % qubit_count])
            x.ctrl(qvector[qubit], qvector[(qubit + 1) % qubit_count])

        # Mixer unitary
        for qubit in range(qubit_count):
            rx(2.0 * thetas[layer + layer_count], qvector[qubit])


# Specify the optimizer and its initial parameters. Make it repeatable.
cudaq.set_random_seed(13)
optimizer = cudaq.optimizers.COBYLA()
np.random.seed(13)
optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0,
                                                 parameter_count)
print("Initial parameters = ", optimizer.initial_parameters)


# Define the objective, return `<state(params) | H | state(params)>`
def objective(parameters):
    return cudaq.observe(kernel_qaoa, hamiltonian, qubit_count, layer_count,
                         parameters).expectation()


# Optimize!
optimal_expectation, optimal_parameters = optimizer.optimize(
    dimensions=parameter_count, function=objective)

# Print the optimized value and its parameters
print("Optimal value = ", optimal_expectation)
print("Optimal parameters = ", optimal_parameters)

# Sample the circuit using the optimized parameters
counts = cudaq.sample(kernel_qaoa, qubit_count, layer_count, optimal_parameters)
print(counts)

Initial parameters =  [0.21810696323572243, -0.20613464375211488, 0.2546877639814583, 0.3657985647468064]
Optimal value =  -1.999998927115975
Optimal parameters =  [1.1187868406824681, -0.5591386753703883, 0.559403603691353, 0.4517996869247546]
{ 0101:521 1010:479 }

